## Limpieza rapida de los archivos Shapefile con limites territoriales tanto de los municipios(colonias) y las alcaldias

In [2]:
import geopandas as gpd
import re
import pandas as pd
from copy_repo_funciones import *
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [8]:
geo= gpd.read_file('../FINAL/RAW/GEOREF/georef-mexico-colonia/georef-mexico-colonia-millesime.shp', encoding='utf-8')

In [14]:
geo.head()

,year,sta_code,sta_name,mun_code,mun_name,col_code,col_name,col_area_co,col_type,geometry
0,2022,09,Ciudad de México,09002,Azcapotzalco,02-009,Coltongo,MEX,colonia,"POLYGON ((-99.15595 19.48651, -99.15294 19.485..."
1,2022,09,Ciudad de México,09003,Coyoacán,03-006,Ampliacion Candelaria,MEX,colonia,"POLYGON ((-99.15265 19.32581, -99.15336 19.326..."
2,2022,09,Ciudad de México,09003,Coyoacán,03-026,Croc Culhuacan Secc 6 (U Hab),MEX,colonia,"POLYGON ((-99.12251 19.32904, -99.12198 19.329..."
3,2022,09,Ciudad de México,09003,Coyoacán,03-072,Los Cipreses,MEX,colonia,"POLYGON ((-99.12466 19.31520, -99.12491 19.315..."
4,2022,09,Ciudad de México,09004,Cuajimalpa de Morelos,04-033,Lomas Del Chamizal,MEX,colonia,"POLYGON ((-99.26654 19.38314, -99.26658 19.383..."


In [10]:
geo['col_name'] = geo['col_name'].str.replace(r"[\[\]']", '', regex=True)
geo['sta_code'] = geo['sta_code'].str.replace(r"[\[\]']", '', regex=True)
geo['sta_namee'] = geo['sta_name'].str.replace(r"[\[\]']", '', regex=True)

In [13]:
geo = geo.apply(lambda x: x.str.replace(r"[\[\]']", '', regex=True) if x.dtype == 'object' else x)

In [16]:
geo.to_file('../FINAL/modified_geo.shp')

C:\Users\anton\AppData\Local\Temp\ipykernel_11828\3943655620.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  geo.to_file('../FINAL/modified_geo.shp')


### TANDEO

In [29]:
tandeo=gpd.read_file('../CLEAN/TABLEAU/tandeo2019.shp', encoding='utf-8')

In [32]:
tandeo.head()

,ID,ENTIDAD,MUNICIPIO,NOMBRE,DELEGACIO,tnd_2019,geometry
0,2059,9,13,DIF,Xochimilco,no,"POLYGON ((-99.09734 19.24389, -99.09614 19.243..."
1,1,9,2,SAN ALVARO,Azcapotzalco,no,"POLYGON ((-99.18803 19.46367, -99.18668 19.463..."
2,1033,9,12,XITLE,Tlalpan,no,"POLYGON ((-99.18350 19.25851, -99.18314 19.258..."
3,2060,9,13,SAN JUAN DE LOS LAGOS(TEZONTITLA),Xochimilco,no,"POLYGON ((-99.11681 19.21855, -99.11656 19.218..."
4,2,9,2,SAN SALVADOR XOCHIMANCA,Azcapotzalco,no,"POLYGON ((-99.17763 19.46195, -99.17699 19.461..."


In [31]:
tandeo['DELEGACIO'] = tandeo['DELEGACIO'].str.title()

In [33]:
tandeo.loc[tandeo['DELEGACIO']=='Coyoacan', 'DELEGACIO']='Coyoacán'
tandeo.loc[tandeo['DELEGACIO']=='Alvaro Obregon', 'DELEGACIO']='Álvaro Obregón'
tandeo.loc[tandeo['DELEGACIO']=='Tlahuac', 'DELEGACIO']='Tláhuac'
tandeo.loc[tandeo['DELEGACIO']=='Benito Juarez', 'DELEGACIO']='Benito Juárez'
tandeo.loc[tandeo['DELEGACIO'] == 'Cuauhtemoc', 'DELEGACIO'] = 'Cuauhtémoc'
tandeo.loc[tandeo['DELEGACIO'] == 'Magdalena Contreras', 'DELEGACIO'] = 'La Magdalena Contreras'

In [47]:
tandeo['DELEGACIO'].value_counts()

DELEGACIO
Iztapalapa                375
Álvaro Obregón            313
Tlalpan                   300
Gustavo A. Madero         281
Tláhuac                   220
Xochimilco                195
Coyoacán                  142
Azcapotzalco              111
Venustiano Carranza        91
Miguel Hidalgo             80
Cuajimalpa de Morelos      65
Benito Juárez              57
La Magdalena Contreras     51
Iztacalco                  47
Milpa Alta                 38
Cuauhtémoc                 33
Name: count, dtype: int64

In [46]:
tandeo.loc[tandeo['DELEGACIO']=='Cuajimalpa De Morelos', 'DELEGACIO']='Cuajimalpa de Morelos'

In [48]:
tandeo.to_file('../CLEAN/TABLEAU/tandeo2019.shp')

### INTENTANDO SACAR POBLACION POR COLONIA

Vale no esta bien este archivo

In [49]:
pob=pd.read_csv('../RAW/CENSOS INEGI/conjunto_de_datos_iter_09CSV20.csv')


In [52]:
pob['NOM_MUN'].value_counts()

NOM_MUN
Milpa Alta                              247
Tlalpan                                 167
Xochimilco                              114
Tláhuac                                  55
Cuajimalpa de Morelos                    27
La Magdalena Contreras                   26
Álvaro Obregón                            9
Total de la entidad Ciudad de México      3
Azcapotzalco                              2
Iztapalapa                                2
Iztacalco                                 2
Gustavo A. Madero                         2
Coyoacán                                  2
Benito Juárez                             2
Cuauhtémoc                                2
Miguel Hidalgo                            2
Venustiano Carranza                       2
Name: count, dtype: int64

### Limpieza archivo escuelas publicas

In [54]:
escuelas=pd.read_csv('../RAW/ESCUELAS/escuelas_publicas.csv', encoding='utf-8')

In [55]:
escuelas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2383 entries, 0 to 2382
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   nombre     2383 non-null   object 
 1   domicilio  2347 non-null   object 
 2   colonia    2231 non-null   object 
 3   alcaldia   2231 non-null   object 
 4   latitud    2242 non-null   float64
 5   longitud   2244 non-null   float64
dtypes: float64(2), object(4)
memory usage: 111.8+ KB


In [ ]:
#limpieza alcaldias

In [57]:
escuelas['alcaldia'] = escuelas['alcaldia'].str.title()

In [59]:
escuelas.loc[escuelas['alcaldia'] == 'Coyoacan', 'alcaldia'] = 'Coyoacán'
escuelas.loc[escuelas['alcaldia'] == 'Alvaro Obregon', 'alcaldia'] = 'Álvaro Obregón'
escuelas.loc[escuelas['alcaldia'] == 'Tlahuac', 'alcaldia'] = 'Tláhuac'
escuelas.loc[escuelas['alcaldia'] == 'Benito Juarez', 'alcaldia'] = 'Benito Juárez'
escuelas.loc[escuelas['alcaldia'] == 'Cuauhtemoc', 'alcaldia'] = 'Cuauhtémoc'
escuelas.loc[escuelas['alcaldia'] == 'Magdalena Contreras', 'alcaldia'] = 'La Magdalena Contreras'
escuelas.loc[escuelas['alcaldia'] == 'Cuajimalpa De Morelos', 'alcaldia'] = 'Cuajimalpa de Morelos'

Usaremos Shapely para sacar las colonias y sus IDS

In [4]:
geo=gpd.read_file('../CLEAN/TABLEAU/modified_geo.shp')

In [62]:
from shapely.geometry import Point, Polygon


for index, row in escuelas.iterrows():
     lat = row['latitud']
     lon = row['longitud']
    

     point = Point(lon, lat)
    

     colonia_name = None
    
 
     for _, polygon in geo.iterrows():

         if polygon['geometry'].contains(point):

             colonia_name = polygon['col_name']
             break 
    
   
     escuelas.at[index, 'Colonia name'] = colonia_name

In [65]:
nancols(escuelas)

colonia         152
alcaldia        152
Colonia name    146
latitud         141
longitud        139
domicilio        36
dtype: int64

In [66]:
escuelas.dropna(subset=['latitud'], inplace=True)


In [68]:
escuelas.dropna(subset=['colonia'], inplace=True)

In [69]:
nancols(escuelas)

domicilio    36
dtype: int64

In [71]:
escuelas['nombre'] = escuelas['nombre'].str.lower()

In [75]:
escuelas['nivel escolar'] = ''  

for index, row in escuelas.iterrows():
    nombre = row['nombre']
    if 'secundaria' in nombre:
        escuelas.at[index, 'nivel escolar'] = 'Secundaria'
    elif 'primaria' in nombre:
        escuelas.at[index, 'nivel escolar'] = 'Primaria'
    elif 'preescolar' in nombre:
        escuelas.at[index, 'nivel escolar'] = 'Preescolar'


In [84]:
nancols(escuelas)

domicilio    36
dtype: int64

In [85]:
col_unique_counts(escuelas, 'nivel escolar')

,nivel escolar,Count
0,Primaria,1134
1,Preescolar,643
2,Secundaria,444
3,,10


In [78]:
escuelas['nombre'] = escuelas['nombre'].str.title()
escuelas['domicilio'] = escuelas['domicilio'].str.title()
escuelas.drop(columns=['colonia'], inplace=True)


In [87]:
#escuelas

In [80]:
escuelas.to_csv('../CLEAN/TABLEAU/escuelas_publicas.csv')

### Limpieza archivo lluvias

In [2]:
lluvia=gpd.read_file('../RAW/atlas_de_riesgo_precipitacion/atlas_de_riesgo_precipitacion.shp')

In [17]:
geo.head()

,year,sta_code,sta_name,mun_code,mun_name,col_code,col_name,col_area_c,col_type,geometry
0,2022,09,Ciudad de México,09002,Azcapotzalco,02-009,Coltongo,MEX,colonia,"POLYGON ((-99.15595 19.48651, -99.15294 19.485..."
1,2022,09,Ciudad de México,09003,Coyoacán,03-006,Ampliacion Candelaria,MEX,colonia,"POLYGON ((-99.15265 19.32581, -99.15336 19.326..."
2,2022,09,Ciudad de México,09003,Coyoacán,03-026,Croc Culhuacan Secc 6 (U Hab),MEX,colonia,"POLYGON ((-99.12251 19.32904, -99.12198 19.329..."
3,2022,09,Ciudad de México,09003,Coyoacán,03-072,Los Cipreses,MEX,colonia,"POLYGON ((-99.12466 19.31520, -99.12491 19.315..."
4,2022,09,Ciudad de México,09004,Cuajimalpa de Morelos,04-033,Lomas Del Chamizal,MEX,colonia,"POLYGON ((-99.26654 19.38314, -99.26658 19.383..."


In [15]:
#lluvia.head()

In [21]:

from shapely.geometry import Point



points = lluvia['g_pnt_2'].apply(lambda x: Point(float(x.split(',')[1]), float(x.split(',')[0])))


lluvia['Colonia'] = None


for idx, point in points.items():
    matched_polygon = None
    for _, polygon in geo.iterrows():
        if polygon['geometry'].contains(point):
            matched_polygon = polygon
            break
    
    
    if matched_polygon is not None:
        lluvia.loc[idx, 'Colonia'] = matched_polygon['col_name']


print(lluvia)


       _id    id                       g_pnt_2   
0        1   817  19.3628767409,-99.2270030874  \
1        2   825   19.359713645,-99.1126636533   
2        3   838  19.3499612368,-99.0618445374   
3        4   844  19.3743243684,-99.0503042071   
4        5   851   19.3232006479,-99.045198103   
5       40   383  19.2284921231,-98.9668113753   
6       41   394  19.3419018187,-99.2540388952   
7       42   403  19.2703936016,-99.1659508925   
8        6   859  19.3480055338,-99.1154810956   
9        7   865  19.3109881499,-99.0899412391   
10       8   871   19.3736559787,-99.056245848   
11       9   893  19.3405068835,-99.0503516331   
12      10   895  19.3288934357,-99.0344088057   
13      11   898  19.3592218638,-99.1255932466   
14      18   103  19.4011587376,-99.1588058018   
15      12   993   19.3464836199,-99.049692185   
16      13   998  19.2921838295,-99.1816848148   
17      14    25  19.3804094569,-99.1487503632   
18      15    53  19.2006471732,-99.1073077931   


In [9]:
#lluvia['Colonia'].value_counts()

In [25]:
lluvia.shape

(4908, 21)

In [26]:
lluvia.to_file('../CLEAN/TABLEAU/lluvia.shp')

In [4]:
lluvia=gpd.read_file('../CLEAN/TABLEAU/lluvia.shp') # se me olvidó adecuar unas cosillas...

#limpiar alcaldias y quitar columnas que no aporten, para aligerar el tableau...

#borrar: id, _id, geo_shp, descrpc, fuente, fenomen, r_p_v_e, taxonom, perd_rt, intens_n, entidad, cvegeo, area_m2, perim_m, 

In [6]:
unique_cols(lluvia)

_id         4908
geo_shp     4908
geometry    4891
g_pnt_2     2455
id          2454
perim_m     2454
area_m2     2454
cvegeo      2454
Colonia     1227
alcaldi       16
intnsdd        5
intns_nm       5
int2           5
descrpc        1
fuente         1
fenomen        1
r_p_v_e        1
taxonom        1
perd_rt        1
intens_n       1
entidad        1
dtype: int64

In [7]:
columns_to_drop = ['id', '_id', 'geo_shp', 'descrpc', 'fuente', 'fenomen', 'r_p_v_e', 'taxonom', 'perd_rt', 'intens_n', 'entidad', 'cvegeo', 'area_m2', 'perim_m']
lluvia.drop(columns=columns_to_drop, inplace=True)


In [8]:
lluvia.head()

,g_pnt_2,intnsdd,alcaldi,intns_nm,int2,Colonia,geometry
0,"19.3628767409,-99.2270030874",Medio,lvaro Obregn,60 a 64,3,Canutillo 3ra Seccion,"POLYGON ((-99.22523 19.36495, -99.22483 19.365..."
1,"19.359713645,-99.1126636533",Bajo,Iztapalapa,54 a 60,2,Escuadron 201,"POLYGON ((-99.11469 19.35688, -99.11525 19.356..."
2,"19.3499612368,-99.0618445374",Bajo,Iztapalapa,54 a 60,2,Constitucion De 1917 I,"POLYGON ((-99.06185 19.35159, -99.06144 19.351..."
3,"19.3743243684,-99.0503042071",Bajo,Iztapalapa,54 a 60,2,Chinampac De Juarez Ii,"POLYGON ((-99.05080 19.37539, -99.05001 19.375..."
4,"19.3232006479,-99.045198103",Muy Bajo,Iztapalapa,41 a 54,1,Desarrollo Urbano Quetzalcoatl Iii,"POLYGON ((-99.04515 19.32252, -99.04526 19.322..."


In [11]:
lluvia.loc[lluvia['alcaldi'] == 'Coyoacn', 'alcaldi'] = 'Coyoacán'
lluvia.loc[lluvia['alcaldi'] == 'lvaro Obregn', 'alcaldi'] = 'Álvaro Obregón'
lluvia.loc[lluvia['alcaldi'] == 'Tlhuac', 'alcaldi'] = 'Tláhuac'
lluvia.loc[lluvia['alcaldi'] == 'Benito Juarez', 'alcaldi'] = 'Benito Juárez'
lluvia.loc[lluvia['alcaldi'] == 'Cuauhtmoc', 'alcaldi'] = 'Cuauhtémoc'



In [13]:
#lluvia['alcaldi'].value_counts()

In [14]:
lluvia.to_file('../CLEAN/TABLEAU/lluvia.shp')

### limpiando colonias de los demas archivos

In [3]:
geo=gpd.read_file('../CLEAN/TABLEAU/modified_geo.shp')

In [11]:
geo.head()

,year,sta_code,sta_name,mun_code,mun_name,col_code,col_name,col_area_c,col_type,geometry
0,2022,09,Ciudad de México,09002,Azcapotzalco,02-009,Coltongo,MEX,colonia,"POLYGON ((-99.15595 19.48651, -99.15294 19.485..."
1,2022,09,Ciudad de México,09003,Coyoacán,03-006,Ampliacion Candelaria,MEX,colonia,"POLYGON ((-99.15265 19.32581, -99.15336 19.326..."
2,2022,09,Ciudad de México,09003,Coyoacán,03-026,Croc Culhuacan Secc 6 (U Hab),MEX,colonia,"POLYGON ((-99.12251 19.32904, -99.12198 19.329..."
3,2022,09,Ciudad de México,09003,Coyoacán,03-072,Los Cipreses,MEX,colonia,"POLYGON ((-99.12466 19.31520, -99.12491 19.315..."
4,2022,09,Ciudad de México,09004,Cuajimalpa de Morelos,04-033,Lomas Del Chamizal,MEX,colonia,"POLYGON ((-99.26654 19.38314, -99.26658 19.383..."


In [4]:
tandeo=gpd.read_file('../CLEAN/TABLEAU/tandeo2019.shp')

In [6]:
tandeo.head()

,ID,ENTIDAD,MUNICIPIO,NOMBRE,DELEGACIO,tnd_2019,geometry
0,2059,9,13,DIF,Xochimilco,no,"POLYGON ((-99.09734 19.24389, -99.09614 19.243..."
1,1,9,2,SAN ALVARO,Azcapotzalco,no,"POLYGON ((-99.18803 19.46367, -99.18668 19.463..."
2,1033,9,12,XITLE,Tlalpan,no,"POLYGON ((-99.18350 19.25851, -99.18314 19.258..."
3,2060,9,13,SAN JUAN DE LOS LAGOS(TEZONTITLA),Xochimilco,no,"POLYGON ((-99.11681 19.21855, -99.11656 19.218..."
4,2,9,2,SAN SALVADOR XOCHIMANCA,Azcapotzalco,no,"POLYGON ((-99.17763 19.46195, -99.17699 19.461..."


In [20]:
from shapely.geometry import Polygon
from shapely.geometry import Point

#calcular centroides de cada polygono para luego cruzar con los pols de geo

points = tandeo['geometry'].centroid
tandeo['info_from_geo'] = None

for idx, point in points.items():
    matched_polygon = None
    for _, polygon in geo.iterrows():
        if polygon['geometry'].contains(point):
            matched_polygon = polygon
            break
    
    if matched_polygon is not None:
        tandeo.at[idx, 'info_from_geo'] = matched_polygon['col_name']

In [21]:
tandeo.head()

,ID,ENTIDAD,MUNICIPIO,NOMBRE,DELEGACIO,tnd_2019,geometry,Colonia name,centroid,Colonia,info_from_geo
0,2059,9,13,DIF,Xochimilco,no,"POLYGON ((-99.09734 19.24389, -99.09614 19.243...",None,POINT (-99.09704 19.24310),,San Lorenzo Atemoaya (Pblo)
1,1,9,2,SAN ALVARO,Azcapotzalco,no,"POLYGON ((-99.18803 19.46367, -99.18668 19.463...",None,POINT (-99.18376 19.46148),,San Alvaro
2,1033,9,12,XITLE,Tlalpan,no,"POLYGON ((-99.18350 19.25851, -99.18314 19.258...",None,POINT (-99.17901 19.25799),,San Andres Totoltepec (Pblo)
3,2060,9,13,SAN JUAN DE LOS LAGOS(TEZONTITLA),Xochimilco,no,"POLYGON ((-99.11681 19.21855, -99.11656 19.218...",None,POINT (-99.11506 19.21789),,Guadalupe
4,2,9,2,SAN SALVADOR XOCHIMANCA,Azcapotzalco,no,"POLYGON ((-99.17763 19.46195, -99.17699 19.461...",None,POINT (-99.17357 19.45956),,San Salvador Xochimanca


In [22]:
nancols(tandeo)

Colonia name     2399
info_from_geo       3
ID                  1
ENTIDAD             1
MUNICIPIO           1
NOMBRE              1
geometry            1
centroid            1
dtype: int64

In [23]:
columns_to_drop = ['ID', 'ENTIDAD', 'MUNICIPIO', 'NOMBRE', 'Colonia name', 'centroid', 'Colonia']
tandeo = tandeo.drop(columns_to_drop, axis=1)
tandeo = tandeo.rename(columns={'info_from_geo': 'Colonia'})

In [24]:
tandeo.head()

,DELEGACIO,tnd_2019,geometry,Colonia
0,Xochimilco,no,"POLYGON ((-99.09734 19.24389, -99.09614 19.243...",San Lorenzo Atemoaya (Pblo)
1,Azcapotzalco,no,"POLYGON ((-99.18803 19.46367, -99.18668 19.463...",San Alvaro
2,Tlalpan,no,"POLYGON ((-99.18350 19.25851, -99.18314 19.258...",San Andres Totoltepec (Pblo)
3,Xochimilco,no,"POLYGON ((-99.11681 19.21855, -99.11656 19.218...",Guadalupe
4,Azcapotzalco,no,"POLYGON ((-99.17763 19.46195, -99.17699 19.461...",San Salvador Xochimanca


In [25]:
tandeo.to_file('../CLEAN/TABLEAU/tandeo2019.shp')

Tandeo finish ahora con inundaciones

In [26]:
inun=gpd.read_file('../CLEAN/TABLEAU/INUNDACIONES.csv')

In [27]:
inun.head()

,cve_ent,alcaldia,cve_col,colonia,pob_2010,SUP_COL_M2,VIV2010,T_AV_M2,M2_AV_HAB,C_M2AVHAB,INUNDACION,INT_NUM_IN,C_INUNDA,S_EPMA_1,C_EPMA_1,geometry
0,09,AZCAPOTZALCO,02-001,AGUILERA,2014,103825,680,5598.16,2.77962,3,Muy Alto,100,5,8,4,None
1,09,AZCAPOTZALCO,02-002,ALDANA,3378,154812,1088,3521.13,1.04237,4,Muy Alto,100,5,9,5,None
2,09,AZCAPOTZALCO,02-005,ANGEL ZIMBRON,2737,242232,1067,16817.3,6.14443,2,Muy Alto,100,5,7,4,None
3,09,AZCAPOTZALCO,02-006,ARENAL,4817,286075,1493,4504.14,0.935051,4,Muy Alto,100,5,9,5,None
4,09,AZCAPOTZALCO,02-007,CENTRO DE AZCAPOTZALCO,3043,424471,1118,52057.6,17.1073,1,Muy Alto,100,5,6,3,None
